Classification of the bacteria data using three different classifiers:
1. SVM
2. RandomForest with decision stumps
3. AdaBoost with decision stumps.

The data is each replicate of each species along the rows, and the fluorescense spectra readings at the lag growth phase along the columns. 

In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
os.getcwd()

'/mnt/d/dev/summer-bursary-2018/bacteria'

In [3]:
os.listdir('data')

['16ms_32ms_growth_phase_spectra.csv',
 '16_ms_lag_codes.csv',
 'bacteria.csv',
 'Classific.py',
 'graphs.pdf',
 'wavelengths.csv']

In [4]:
df = pd.read_csv('data/bacteria.csv', header=[0, 1, 2], index_col=0)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1043 entries, 0 to 1042
Columns: 135 entries, (lag, bc, 01) to (stat, se, 07)
dtypes: float64(135)
memory usage: 1.1 MB


# Preprocessing

In [5]:
lag = df['lag'].T
lag

0         1         2          3         4     \
species replicate                                                      
bc      01         0.000000  1.560060  0.715025  -0.520060  0.130014   
        02         0.622353 -2.115990  0.062234   2.178335  1.867163   
        03         0.064528  0.645279 -1.032450  -2.064991 -1.419711   
        04        -1.094650 -2.554170 -0.912210   0.851433  3.709871   
        05        -1.492810 -0.559800  2.923387   3.047967  2.985793   
        06        -0.979760 -0.061230  0.489875  -1.530940  0.000003   
        07        -0.240070  0.600163  0.540146  -1.200400  1.260425   
        08         0.060945 -0.792280  0.182837   0.853271 -3.047420   
        09         0.247423  0.123709 -0.742270   1.360885  2.226921   
        10         2.628512  0.292028 -0.408840  -5.741500  4.453517   
        11         0.637664  1.785430  0.191293  -0.127530 -0.956540   
        12         0.192628 -0.192630  0.577884  -0.192640 -0.321060   
ec      01        -0.849660  0.849660  2.352898  -0.457530  0.065363   
        02         0.786579 -1.114310  0.721023   1.638780  1.507702   
        03        -0.129750  0.648743 -0.713620  -1.427320  0.908299   
        04        -0.642560 -1.986080  0.116830   1.343595  1.518857   
        05        -0.597650  0.179290  0.657402   0.717218  1.793046   
        06        -1.056270  0.248530 -0.186400  -2.733990  0.124277   
        07         3.554267  1.881335 -0.557560 -13.423800 -3.356780   
        08         0.188377 -1.318620  0.062791  -1.193100  0.753540   
lm      01        -3.504790  4.778309  6.435558   1.725630  0.191804   
        02         0.177790 -0.533370 -0.474100  -1.778000 -0.177810   
        03         0.519307 -0.259650 -0.259650   2.077325  1.038670   
        04        -0.776180  0.776173  0.323411  -0.776220  1.293722   
        05        -0.061520  1.107426 -0.123050   0.184587 -0.861390   
pa      01        -0.250260 -0.062570  1.376439  -0.813400 -0.500560   
        02        -1.198300  1.654795 -1.940110   1.312503 -0.970120   
        03        -0.124480  0.373440  1.431539   1.929570  2.614285   
        04        -0.397470  0.066244  0.463717   1.126222  0.927487   
        05        -0.322480  0.515967 -0.193490  -1.354500 -0.387000   
        06        -1.641770  0.117268 -0.234540   0.645014 -0.293190   
sa      01         0.141226 -0.494290  2.118387  -0.494320  0.776791   
        02        -0.922270  0.368908 -1.106730   1.106783  0.799366   
        03        -0.375890 -0.438540  0.187944   3.007268  1.691599   
        04        -0.303680 -0.364410 -1.275440   1.579194  0.121470   
        05        -1.180120  0.413041  0.413041   0.767116  1.416233   
        06         0.933263 -0.933260 -1.399900  -0.583320  0.349998   
        07         1.115037 -0.065600 -1.246210  -0.459150 -0.787120   
        08         0.186163 -1.116970  0.682596   1.737603  0.372347   
        09        -0.765110 -0.637480 -1.594040  -7.227790  9.661653   
        10        -0.911120 -0.607410  1.397044   0.850412  0.060745   
        11        -0.851670 -0.365000 -0.304170  -1.216730  0.365026   
se      01         0.128596  1.157322  0.964434  -0.064300  0.192902   
        02         0.328784 -0.131510 -0.460300  -1.052160 -0.526080   
        03         1.344575 -1.216520  0.256110   0.640305  0.384181   
        04         2.090057 -0.809050 -1.415850   0.471977  0.471977   
        05        -1.547500 -1.190380  0.238074   0.833313 -1.547610   

                        5             6         7          8             9     \
species replicate                                                               
bc      01         -2.080180  0.000000e+00  1.040112   1.235136 -1.950200e-01   
        02          1.120305  8.713430e-01 -1.369260  -1.929420  6.846220e-01   
        03          1.032515 -2.000491e+00 -0.903453   2.968462  1.032499e+00   
        04          1.520432  8.514440e-01 -2.067810  -1.581270  3.040850e-01   
        

In [6]:
lag.isnull().values.any()

False

In [7]:
X = lag

In [8]:
y = X.reset_index()['species']
y.head()

0    bc
1    bc
2    bc
3    bc
4    bc
Name: species, dtype: object

In [9]:
y.describe()

count     47
unique     6
top       bc
freq      12
Name: species, dtype: object

In [10]:
print(y)

0     bc
1     bc
2     bc
3     bc
4     bc
5     bc
6     bc
7     bc
8     bc
9     bc
10    bc
11    bc
12    ec
13    ec
14    ec
15    ec
16    ec
17    ec
18    ec
19    ec
20    lm
21    lm
22    lm
23    lm
24    lm
25    pa
26    pa
27    pa
28    pa
29    pa
30    pa
31    sa
32    sa
33    sa
34    sa
35    sa
36    sa
37    sa
38    sa
39    sa
40    sa
41    sa
42    se
43    se
44    se
45    se
46    se
Name: species, dtype: object


In [11]:
from sklearn.utils import shuffle

X, y = shuffle(X, y)

print(X[:5])
print(y[:5])

                       0         1         2         3         4     \
species replicate                                                     
bc      03         0.064528  0.645279 -1.032450 -2.064991 -1.419711   
pa      06        -1.641770  0.117268 -0.234540  0.645014 -0.293190   
lm      01        -3.504790  4.778309  6.435558  1.725630  0.191804   
        03         0.519307 -0.259650 -0.259650  2.077325  1.038670   
sa      10        -0.911120 -0.607410  1.397044  0.850412  0.060745   

                        5         6         7         8             9     \
species replicate                                                          
bc      03          1.032515 -2.000491 -0.903453  2.968462  1.032499e+00   
pa      06          0.996838 -0.820940  2.052344 -0.938200 -2.110960e+00   
lm      01         12.464440 -4.282870 -1.150720  2.748759 -7.350220e+00   
        03          0.259666 -2.207160  0.519336 -1.103600  2.140000e-07   
sa      10         -2.065330 -0.789690 -2.6120

# PCA + SVM

In [12]:
from sklearn.decomposition import PCA

pca = PCA(n_components=0.99, svd_solver='full')
pca.fit(X)

X_pca = pca.transform(X)
print(X_pca.shape)

(47, 2)


In [13]:
from sklearn.svm import SVC
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, cross_val_score, GridSearchCV

param_grid = {
    'kernel': ['linear', 'rbf'],
    'gamma': [10 ** -n for n in range(6)],
    'C': [10 ** n for n in range(-5, 2)]
}

cv = StratifiedKFold(n_splits=3)
clf = SVC()

grid_search = GridSearchCV(clf, param_grid, cv=cv, verbose=10, n_jobs=4)
grid_search.fit(X_pca, y)

print(grid_search.best_score_)
print(grid_search.best_params_)

Fitting 3 folds for each of 84 candidates, totalling 252 fits
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=linear .................................
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV]  C=1e-05, gamma=1, kernel=linear, score=0.5882352941176471, total=   0.0s
[CV]  C=1e-05, gamma=1, kernel=linear, score=0.5294117647058824, total=   0.0s
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV] C=1e-05, gamma=1, kernel=rbf ....................................
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=1e-05, gamma=0.1, kernel=linear ...............................
[CV] C=1e-05, gamma=0.1, kernel=linear ...............................
[CV]  C=1e-05, gamma=1, kernel=rbf, score=0.23076923076923

[Parallel(n_jobs=4)]: Batch computation too fast (0.0470s.) Setting batch_size=8.
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Batch computation too fast (0.0739s.) Setting batch_size=42.
[Parallel(n_jobs=4)]: Done  24 tasks      | elapsed:    0.1s


[CV]  C=0.0001, gamma=1, kernel=linear, score=0.6153846153846154, total=   0.1s
[CV] C=0.0001, gamma=1, kernel=rbf ...................................
[CV]  C=0.0001, gamma=0.001, kernel=linear, score=0.6153846153846154, total=   0.1s
[CV] C=0.0001, gamma=0.001, kernel=rbf ...............................
[CV]  C=0.0001, gamma=1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.0001, gamma=0.0001, kernel=rbf ..............................
[CV]  C=0.0001, gamma=0.001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.0001, gamma=0.001, kernel=rbf ...............................
[CV]  C=0.0001, gamma=0.0001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.0001, gamma=0.0001, kernel=rbf ..............................
[CV]  C=0.0001, gamma=0.001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.0001, gamma=0.001, kernel=rbf ...............................
[CV]  C=0.0001, gamma=0.0001, kernel=rbf, score=0.3076923076923077, total=   0.0s
[CV

[CV]  C=0.001, gamma=0.0001, kernel=linear, score=0.5294117647058824, total=   0.1s
[CV] C=0.001, gamma=0.0001, kernel=linear ............................
[CV]  C=0.001, gamma=0.0001, kernel=linear, score=0.5882352941176471, total=   0.4s
[CV] C=0.001, gamma=0.0001, kernel=linear ............................
[CV]  C=0.001, gamma=0.0001, kernel=linear, score=0.6153846153846154, total=   1.0s
[CV] C=0.001, gamma=0.0001, kernel=rbf ...............................
[CV]  C=0.001, gamma=0.0001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.001, gamma=0.0001, kernel=rbf ...............................
[CV]  C=0.001, gamma=0.0001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=0.001, gamma=0.0001, kernel=rbf ...............................
[CV]  C=0.001, gamma=0.0001, kernel=rbf, score=0.3076923076923077, total=   0.0s
[CV] C=0.001, gamma=1e-05, kernel=linear .............................
[CV]  C=0.001, gamma=1e-05, kernel=linear, score=0.5294117647058824, total=  

[CV] C=0.1, gamma=1, kernel=linear ...................................
[CV]  C=0.1, gamma=0.001, kernel=linear, score=0.5294117647058824, total=   7.4s
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV]  C=0.1, gamma=0.001, kernel=linear, score=0.47058823529411764, total=  19.5s
[CV] C=0.1, gamma=0.001, kernel=linear ...............................
[CV]  C=1, gamma=0.001, kernel=linear, score=0.5294117647058824, total= 1.8min
[CV] C=1, gamma=0.001, kernel=linear .................................
[CV]  C=1, gamma=0.001, kernel=linear, score=0.6153846153846154, total=  53.3s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV]  C=1, gamma=0.001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV]  C=1, gamma=0.001, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=1, gamma=0.001, kernel=rbf ....................................
[CV]  C=1, gamma=0.001, kern

[CV]  C=1, gamma=1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=1, gamma=1, kernel=rbf ........................................
[CV]  C=1, gamma=1, kernel=rbf, score=0.23076923076923078, total=   0.0s
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV]  C=1, gamma=0.1, kernel=linear, score=0.5294117647058824, total=  54.0s
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV]  C=1, gamma=0.1, kernel=linear, score=0.5294117647058824, total= 1.4min
[CV] C=1, gamma=0.1, kernel=linear ...................................
[CV]  C=10, gamma=1e-05, kernel=linear, score=0.5294117647058824, total= 5.0min
[CV] C=10, gamma=1e-05, kernel=linear ................................
[CV]  C=1, gamma=0.1, kernel=linear, score=0.6153846153846154, total=  45.7s
[CV] C=1, gamma=0.1, kernel=rbf ......................................
[CV]  C=1, gamma=0.1, kernel=rbf, score=0.23529411764705882, total=   0.0s
[CV] C=1, gamma=0.1, kernel=rbf ..........

[Parallel(n_jobs=4)]: Done 252 out of 252 | elapsed: 48.5min finished


In [14]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.54 (+/- 0.18)
CPU times: user 219 ms, sys: 0 ns, total: 219 ms
Wall time: 212 ms


In [15]:
%%time

cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(grid_search.best_estimator_, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.57 (+/- 0.23)
CPU times: user 375 ms, sys: 15.6 ms, total: 391 ms
Wall time: 375 ms


# Random Forest with Decision Stumps

In [16]:
%%time

from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

/home/anthony/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


Accuracy: 0.44 (+/- 0.12)
CPU times: user 44.3 s, sys: 359 ms, total: 44.7 s
Wall time: 44.7 s


In [17]:
%%time

clf = RandomForestClassifier(n_estimators=1024, max_depth=1)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.47 (+/- 0.10)
CPU times: user 46.3 s, sys: 328 ms, total: 46.6 s
Wall time: 46.6 s


# AdaBoost

In [18]:
%%time

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X_pca, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.35 (+/- 0.18)
CPU times: user 50.7 s, sys: 0 ns, total: 50.7 s
Wall time: 50.7 s


In [19]:
%%time

clf = AdaBoostClassifier(DecisionTreeClassifier(max_depth=1),
                        n_estimators=1024)
cv = RepeatedStratifiedKFold(n_splits=3, n_repeats=20)
scores = cross_val_score(clf, X, y, cv=cv)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.38 (+/- 0.17)
CPU times: user 2min 40s, sys: 0 ns, total: 2min 40s
Wall time: 2min 40s
